# GENERATE FIGURE INPUTS

This notebook provides functions and scripts for generating input files for visualization with D3.

---
- [WORKSPACE VARIABLES](#workspace-variables)
- **[SITE ARCHITECTURE](#site-architecture)**
    - [point distances](#site-architecture-point-distances)
    - [type grid](#site-architecture-type-grid)
- **[ESTIMATED HEMODYNAMICS](#estimated-hemodynamics)**
- **[EXACT HEMODYNAMICS](#exact-hemodynamics)**
- **[VASCULAR DAMAGE](#vascular-damage)**
- **[VASCULAR FUNCTION](#vascular-function)**
---

Each of the simulation sets has a corresponding script of the same name, which contains a class (of the same name) with relevant condition variables and methods to iterate through these condition.
The `loop` method will be the most often, to iterate through all conditions, extract relevant information, and then compile the information into a single output file.
The `loop` method works with both individual files for each condition (`.csv` or `.json`) or a `.tar.xz` compressed archive of the files for each condition.
For a given figure, not all conditions may be used.

All generated figure inputs are provided in the `analysis` directory.
As much as possible, files are provided in compressed archive form, using the provided `compress.sh` script, taking three arguments: `[NAME OF SIMULATION SET] [EXTENSION NAME] [json OR csv]`.
The corresponding arguments for each of the following figures is provided in the description.

The `generate.py` file contains functions to generate figure inputs.

In [ ]:
from scripts.generate import *

<a class="anchor" id="workspace-variables"></a>

### WORKSPACE VARIABLES

Set up workspace variables for analyzing simulations.

- **`DATA_PATH`** is the path to data files (`.tar.xz` files of compressed simulation outputs)
- **`RESULTS_PATH`** is the path to result files (`.pkl` files generated by parsing)
- **`ANALYSIS_PATH`** is the path for analysis files (`.json` and `.csv` files, `.tar.xz` compressed archives)

In [ ]:
DATA_PATH = "/path/to/data/files/"
RESULTS_PATH = "/path/to/result/files/"
ANALYSIS_PATH = "/path/to/analysis/files/"

<a class="anchor" id="site-architecture"></a>

### SITE ARCHITECTURE

Generate `SITE_ARCHITECTURE` figure inputs.

In [ ]:
from scripts.SITE_ARCHITECTURE import SITE_ARCHITECTURE

<a class="anchor" id="site-architecture-point-distances"></a>

**point distances**

The point distances figure shows the maximum distance of cells from the center of the simulation as a function of point source distance from the center.

The function `make_point_distances` calculates this distance for each of the point sources.
These files are then merged into a single file `SITE_ARCHITECTURE.DISTANCES.csv` that is used as input for D3.
Files created by the function `make_point_distances` can be compressed with `compress.sh` using `SITE_ARCHITECTURE .DISTANCES csv`.

In [ ]:
sites = ["SOURCE"]
layouts = {
    "SOURCE": [
        "point0","point2","point4","point6","point8","point10",
        "point12","point14","point16","point18","point20",
        "point22","point24","point26","point28","point30",
        "point32","point34","point36","point38",
    ]
}

SITE_ARCHITECTURE.run(ANALYSIS_PATH, RESULTS_PATH, make_point_distances, timepoints=[None], \
                      sites=sites, layouts=layouts)
SITE_ARCHITECTURE.loop(ANALYSIS_PATH, merge_point_distances, save_point_distances, ".DISTANCES", \
                       timepoints=[None], sites=sites, layouts=layouts)

<a class="anchor" id="site-architecture-state-grid"></a>

**type grid**

The type grid figure shows cell type distribution across the colony for different combinations of source grid spacing in the horizontal and vertical directions.

The function `make_type_grid_borders` extracts this farthest border for each of the grid sources across seeds.
These files are then merged into a single file `SITE_ARCHITECTURE.BORDERS.csv` that is used as input for D3.
Files created by the function `make_type_grid_borders` can be compressed with `compress.sh` using `SITE_ARCHITECTURE.BORDERS csv`.

The `.LOCATIONS` files produced in the basic analysis step are also used as input.
Make sure to EXTRACT THE ARCHIVE FILES from `SITE_ARCHITECTURE.LOCATIONS.tar.xz`.

In [ ]:
sites = ["SOURCE"]
layouts = {
    "SOURCE": [
        "constant","grid2","grid3","grid4","grid5",
        "x1y2","x1y3","x1y4","x1y5",
        "x2y1","x2y3","x2y4","x2y5",
        "x3y1","x3y2","x3y4","x3y5",
        "x4y1","x4y2","x4y3","x4y5",
        "x5y1","x5y2","x5y3","x5y4",
    ]
}

SITE_ARCHITECTURE.run(ANALYSIS_PATH, RESULTS_PATH, make_type_grid_borders, timepoints=[30], \
                      sites=sites, layouts=layouts)
SITE_ARCHITECTURE.loop(ANALYSIS_PATH, merge_type_grid_borders, save_type_grid_borders, ".BORDERS", \
                       timepoints=[None], sites=sites, layouts=layouts)

<a class="anchor" id="estimated-hemodynamics"></a>

### ESTIMATED HEMODYNAMICS

Generate `ESTIMATED_HEMODYNAMICS` figure inputs.

In [ ]:
from scripts.ESTIMATED_HEMODYNAMICS import ESTIMATED_HEMODYNAMICS

<a class="anchor" id="exact-hemodynamics"></a>

### EXACT HEMODYNAMICS

Generate `EXACT_HEMODYNAMICS` figure inputs.

In [ ]:
from scripts.EXACT_HEMODYNAMICS import EXACT_HEMODYNAMICS

<a class="anchor" id="vascular-damage"></a>

### VASCULAR DAMAGE

Generate `VASCULAR_DAMAGE` figure inputs.

In [ ]:
from scripts.VASCULAR_DAMAGE import VASCULAR_DAMAGE

<a class="anchor" id="vascular-function"></a>

### VASCULAR FUNCTION

Generate `VASCULAR_FUNCTION` figure inputs.

In [ ]:
from scripts.VASCULAR_FUNCTION import VASCULAR_FUNCTION